프로젝트 : mini BERT 만들기
===

vocab size를 8000으로 줄이고, 전체 파라미터 사이즈가 1M 정도가 되는 아주 작은 mini BERT 모델을 만들어 10Epoch까지 학습시킨 모델을 만들어 봅니다. 또한, 학습 진행 결과를 시각화하여 확인해 봅니다. 

[목차]
1. Tokenizer 준비
2. 데이터 전처리 (1) MASK 생성
3. 데이터 전처리 (2) NSP pair 생성
4. 데이터 전처리 (3) 데이터셋 완성
5. BERT 모델 구현
6. pretrain 진행
7. 프로젝트 결과


In [1]:
# 모듈 imports
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow.keras.backend as K

import os
import re
import math
import numpy as np
import pandas as pd
import random
import collections
import json
import shutil
import zipfile
import copy
from datetime import datetime

import matplotlib.pyplot as plt
import sentencepiece as spm
from tqdm.notebook import tqdm

random_seed = 1234
random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

# tf version 및 gpu 확인
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
print(tf.test.gpu_device_name())

2.4.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
/device:GPU:0


# 1. Tokenizer 준비

먼저, SentencePiece 기반의 토크나이저를 준비합니다. 
- [SentencePiece](https://github.com/google/sentencepiece)

준비해 둔 한글 나무위키 코퍼스로부터 8000의 vocab_size를 갖는 sentencepiece 모델을 생성합니다. 

In [ ]:
import sentencepiece as spm
import os
corpus_file = os.getenv('HOME')+'/aiffel/GoingDeeper/nlp14/data/kowiki.txt'
prefix = 'ko_32000'
vocab_size = 8000

spm.SentencePieceTrainer.train(
    f"--input={corpus_file} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의 토큰

In [ ]:
data_dir = os.getenv('HOME')+'/aiffel/GoingDeeper/nlp14/data'
model_dir = os.getenv('HOME')+'/aiffel/GoingDeeper/nlp14/models'

# vocab loading
vocab = spm.SentencePieceProcessor()
vocab.load(f"{model_dir}/ko_8000.model")

토크나이저가 잘 만들어졌는지 확인해 봅니다. 

In [ ]:
# 특수 token 7개를 제외한 나머지 tokens 들
vocab_list = []
for id in range(7, len(vocab)):
    if not vocab.is_unknown(id):
        vocab_list.append(vocab.id_to_piece(id))

In [ ]:
print(vocab_list[:20])